In [1]:
#Import Libraries
import requests
from bs4 import BeautifulSoup
import datetime
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [2]:
# URL of the CNBC finance news page
url = 'https://www.cnbc.com/stocks/'

# Make the request to the webpage
response = requests.get(url)

# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(response.text, 'html.parser')
# Find all the headline elements on the page
headlines = soup.find_all('a', class_='Card-title')

# Get the current timestamp
timestamp =  datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')

# Print the headlines and save the timestamp
for headline in headlines:
    print(headline.text.strip())
print('Timestamp:', timestamp)
#text.append(headlines)



Alibaba and more: Morgan Stanley names 5 global stocks with at least 50% upside
Delta is the most overbought stock on Wall Street right now. Here are the others
This compact fund beats the market and provides a downside cushion
Blinken meets Chinese Foreign Minister Qin Gang in high-stakes diplomatic trip
The Japanese equity rally could broaden out. Some small caps may get a boost
The IPO market may put pressure on the Fed, Cramer says
Wait for a price break next week while the market's overbought, Jim Cramer says
Jim Cramer explains why Domino's Pizza stock soared this week
Cramer's Lightning Round: Dominion Energy is "a little too risky"
Nvidia gets price target hike amid 10% weekly gain. Here's what the pros say
Toyota defies skeptics as stock seals best week since 2009
These two cruise stocks and this chipmaker led the market higher this week
Stocks making the biggest moves midday: Virgin Galactic, iRobot, Cava and more
The reshoring boom is stoking demand for robotics. Here are th

In [3]:
df = pd.DataFrame(headlines)

In [4]:
df.head(10)

,0
0,Alibaba and more: Morgan Stanley names 5 globa...
1,Delta is the most overbought stock on Wall Str...
2,This compact fund beats the market and provide...
3,Blinken meets Chinese Foreign Minister Qin Gan...
4,The Japanese equity rally could broaden out. S...
5,"The IPO market may put pressure on the Fed, Cr..."
6,Wait for a price break next week while the mar...
7,Jim Cramer explains why Domino's Pizza stock s...
8,"Cramer's Lightning Round: Dominion Energy is ""..."
9,Nvidia gets price target hike amid 10% weekly ...


In [5]:
df.columns = ['headline']

In [6]:
df.head()

,headline
0,Alibaba and more: Morgan Stanley names 5 globa...
1,Delta is the most overbought stock on Wall Str...
2,This compact fund beats the market and provide...
3,Blinken meets Chinese Foreign Minister Qin Gan...
4,The Japanese equity rally could broaden out. S...


In [7]:
df.shape  # Checking number of rows

(35, 1)

__Sentiment Analysis__

In [8]:
from textblob import TextBlob  #Import TextBlob Library for Sentiment Analysis

#Calculate Sentiment Polarity Scores and Save as Sentiment Score
df['sentiment_score'] = df['headline'].apply(lambda x: TextBlob(x).sentiment.polarity)

#Classify Sentiments as positive, negative and neutral
df['sentiment'] = df['sentiment_score'].apply(lambda x: 'negative' if x < -0.2 else ('neutral' if -0.2 <= x <= 2 else 'positive'))


In [9]:
df.head()

,headline,sentiment_score,sentiment
0,Alibaba and more: Morgan Stanley names 5 globa...,0.066667,neutral
1,Delta is the most overbought stock on Wall Str...,0.392857,neutral
2,This compact fund beats the market and provide...,0.000000,neutral
3,Blinken meets Chinese Foreign Minister Qin Gan...,-0.062500,neutral
4,The Japanese equity rally could broaden out. S...,-0.125000,neutral


___Checking for Relevant Words___

In [10]:
#Creat a copy of the dataframe to work with
df_new = df.copy()

In [11]:
import nltk
stopwords = nltk.corpus.stopwords.words('english')
stopwords[0:10]
['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]

In [12]:
def remove_stopwords(text):
    output= " ".join(i for i in text if i not in stopwords)
    return output

In [13]:
import spacy

nlp = spacy.load('en_core_web_sm')

def get_relevant_words(text):
    doc = nlp(text)
    relevant_words = []
    for token in doc:
        if not token.is_stop and token.is_alpha:
            relevant_words.append(token.lemma_.lower())
    return relevant_words

df_new['relevant_words'] = df_new['headline'].apply(get_relevant_words)


In [14]:
df_new.head()

,headline,sentiment_score,sentiment,relevant_words
0,Alibaba and more: Morgan Stanley names 5 globa...,0.066667,neutral,"[alibaba, morgan, stanley, name, global, stock..."
1,Delta is the most overbought stock on Wall Str...,0.392857,neutral,"[delta, overbought, stock, wall, street, right]"
2,This compact fund beats the market and provide...,0.000000,neutral,"[compact, fund, beat, market, provide, downsid..."
3,Blinken meets Chinese Foreign Minister Qin Gan...,-0.062500,neutral,"[blinken, meet, chinese, foreign, minister, qi..."
4,The Japanese equity rally could broaden out. S...,-0.125000,neutral,"[japanese, equity, rally, broaden, small, cap,..."


___Translation___

In [15]:
from googletrans import Translator

translator = Translator()

# Translate headlines to Spanish
df_new['headline_spanish'] = df_new['headline'].apply(lambda x: translator.translate(x, src = 'en', dest='es').text)

# Translate headlines to English
df_new['headline_Italian'] = df_new['headline'].apply(lambda x: translator.translate(x, src = 'en', dest='it').text)


In [16]:
df_new.head()

,headline,sentiment_score,sentiment,relevant_words,headline_spanish,headline_Italian
0,Alibaba and more: Morgan Stanley names 5 globa...,0.066667,neutral,"[alibaba, morgan, stanley, name, global, stock...",Alibaba y más: Morgan Stanley nombra 5 accione...,Alibaba e non solo: Morgan Stanley nomina 5 ti...
1,Delta is the most overbought stock on Wall Str...,0.392857,neutral,"[delta, overbought, stock, wall, street, right]",Delta es la acción más sobrecomprada en Wall S...,Delta è il titolo più ipercomprato di Wall Str...
2,This compact fund beats the market and provide...,0.000000,neutral,"[compact, fund, beat, market, provide, downsid...",Este fondo compacto supera al mercado y propor...,Questo fondo compatto batte il mercato e forni...
3,Blinken meets Chinese Foreign Minister Qin Gan...,-0.062500,neutral,"[blinken, meet, chinese, foreign, minister, qi...",Blinken se reúne con el ministro de Relaciones...,Blinken incontra il ministro degli Esteri cine...
4,The Japanese equity rally could broaden out. S...,-0.125000,neutral,"[japanese, equity, rally, broaden, small, cap,...",El repunte de las acciones japonesas podría ex...,Il rally dell'azionario giapponese potrebbe am...


In [17]:
# Add column for URLs using a lambda function
df_new['url'] = list(map(lambda x: x['href'], headlines))

# Add a column for the timestamp using a lambda function
df_new['Timestamp'] = timestamp

df_new.head()


,headline,sentiment_score,sentiment,relevant_words,headline_spanish,headline_Italian,url,Timestamp
0,Alibaba and more: Morgan Stanley names 5 globa...,0.066667,neutral,"[alibaba, morgan, stanley, name, global, stock...",Alibaba y más: Morgan Stanley nombra 5 accione...,Alibaba e non solo: Morgan Stanley nomina 5 ti...,https://www.cnbc.com/2023/06/19/alibaba-and-mo...,2023-06-18 21:43:18
1,Delta is the most overbought stock on Wall Str...,0.392857,neutral,"[delta, overbought, stock, wall, street, right]",Delta es la acción más sobrecomprada en Wall S...,Delta è il titolo più ipercomprato di Wall Str...,https://www.cnbc.com/2023/06/18/delta-is-the-m...,2023-06-18 21:43:18
2,This compact fund beats the market and provide...,0.000000,neutral,"[compact, fund, beat, market, provide, downsid...",Este fondo compacto supera al mercado y propor...,Questo fondo compatto batte il mercato e forni...,https://www.cnbc.com/2023/06/18/this-compact-f...,2023-06-18 21:43:18
3,Blinken meets Chinese Foreign Minister Qin Gan...,-0.062500,neutral,"[blinken, meet, chinese, foreign, minister, qi...",Blinken se reúne con el ministro de Relaciones...,Blinken incontra il ministro degli Esteri cine...,https://www.cnbc.com/2023/06/18/blinken-meets-...,2023-06-18 21:43:18
4,The Japanese equity rally could broaden out. S...,-0.125000,neutral,"[japanese, equity, rally, broaden, small, cap,...",El repunte de las acciones japonesas podría ex...,Il rally dell'azionario giapponese potrebbe am...,https://www.cnbc.com/2023/06/17/the-japanese-e...,2023-06-18 21:43:18


In [18]:
#Save the data to a CSV file
filename = f"headlines_data.csv"
df.to_csv(filename)
    

To extract historical stock prices using the Alpha Vantage API, you will need an API key, which you can obtain for free by signing up on the Alpha Vantage website. Once you have your API key, you can make requests to the API endpoint to retrieve the desired stock market data. Here's an example of how you can use the Alpha Vantage API to get historical stock prices for the following companies: Apple (AAPL), Microsoft (MSFT), and Amazon (AMZN).

Make sure to replace "YOUR_API_KEY" with your actual API key obtained from the Alpha Vantage website. The code above will fetch the full daily time series data for each symbol and print the open, high, low, close, and volume values for each date.

***ALPHAVANTAGE***

In [19]:
import requests
import pandas as pd

# Set your Alpha Vantage API key
api_key = "XWXPKKBU1W1HEQ4L"

# Define the companies and their respective symbols
companies = {
    "Apple": "AAPL",
    "Microsoft": "MSFT",
    "Google": "GOOGL",
    "Amazon": "AMZN",
    "Meta": "META"  # Assuming "Meta" refers to Facebook (FB)
}

# Define the start and end dates for the historical data
start_date = "2021-01-01"
end_date = "2022-12-31"

# Loop through each company and retrieve the historical stock prices
for company, symbol in companies.items():
    # Construct the API URL
    url = f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol={symbol}&outputsize=full&apikey={api_key}"
    
    # Send the API request
    response = requests.get(url)
    
    # Convert the response to a DataFrame
    data = response.json()["Time Series (Daily)"]
    #data = response.json()['Times Series(INTRADAY)']
    df_stocks = pd.DataFrame(data).T
    
    # Set the column names
    df_stocks.columns = ["open_price", "highest_price", "low_price", "close_price", "adjusted_close", "volume", "Dividend Amount", 'Split_ration']
    
    # Filter the DataFrame based on the specified date range
    df_stocks = df_stocks[(df_stocks.index >= start_date) & (df_stocks.index <= end_date)]
    
    #Save the data to a CSV file
    filename = f"{company}_stock_prices.csv"
    df_stocks.to_csv(filename)
    
    print(f"Stock prices for {company} saved to {filename}.")
    
    

Stock prices for Apple saved to Apple_stock_prices.csv.
Stock prices for Microsoft saved to Microsoft_stock_prices.csv.
Stock prices for Google saved to Google_stock_prices.csv.
Stock prices for Amazon saved to Amazon_stock_prices.csv.
Stock prices for Meta saved to Meta_stock_prices.csv.


In [20]:
df_stocks.head()

,open_price,highest_price,low_price,close_price,adjusted_close,volume,Dividend Amount,Split_ration
2022-12-30,118.16,120.42,117.74,120.34,120.34,19583825,0.0000,1.0
2022-12-29,116.4,121.03,115.77,120.26,120.26,22366192,0.0000,1.0
2022-12-28,116.25,118.15,115.51,115.62,115.62,19612473,0.0000,1.0
2022-12-27,117.93,118.6,116.0501,116.88,116.88,21392311,0.0000,1.0
2022-12-23,116.03,118.175,115.535,118.04,118.04,17796625,0.0000,1.0


In [22]:
# Drop the specified columns
df_stocks = df_stocks.drop(columns=[ "Split_ration", "Dividend Amount"])

In [23]:
df_stocks

,open_price,highest_price,low_price,close_price,adjusted_close,volume
2022-12-30,118.16,120.42,117.74,120.34,120.34,19583825
2022-12-29,116.4,121.03,115.77,120.26,120.26,22366192
2022-12-28,116.25,118.15,115.51,115.62,115.62,19612473
2022-12-27,117.93,118.6,116.0501,116.88,116.88,21392311
2022-12-23,116.03,118.175,115.535,118.04,118.04,17796625
...,...,...,...,...,...,...
2021-01-08,268.31,268.95,263.18,267.57,267.57,18528251
2021-01-07,265.9,271.61,264.78,268.74,268.74,15789756
2021-01-06,262.0,267.75,260.01,263.31,263.31,24354149
2021-01-05,268.29,272.4,268.2111,270.97,270.97,9871557


In [24]:
#  Add a column for symbol for each data set
df1 = pd.read_csv("Apple_stock_prices.csv")
df1["symbol"] = 'AAPL'

df2 = pd.read_csv("Microsoft_stock_prices.csv")
df2["symbol"] = 'MSFT'

df3 = pd.read_csv("Google_stock_prices.csv")
df3["symbol"] = 'GOOGL'

df4 = pd.read_csv("Amazon_stock_prices.csv")
df4["symbol"] = 'AMZN'

df4 = pd.read_csv("Meta_stock_prices.csv")
df4["symbol"] = 'META'


In [25]:
df_stocks = pd.concat([df1, df2, df3, df4])

In [26]:
df_stocks.sample(10) 

,Unnamed: 0,open_price,highest_price,low_price,close_price,adjusted_close,volume,Dividend Amount,Split_ration,symbol
232,2022-01-28,300.23,308.48,294.450,308.26,303.936365,49743698,0.0,1.0,MSFT
492,2021-01-19,213.75,216.98,212.630,216.44,211.631362,30480859,0.0,1.0,MSFT
219,2022-02-16,2732.93,2761.72,2698.475,2754.76,137.738000,1251065,0.0,1.0,GOOGL
486,2021-01-27,1874.91,1880.47,1797.280,1818.94,90.947000,4125631,0.0,1.0,GOOGL
169,2022-04-29,288.61,289.88,276.500,277.52,274.193963,37073923,0.0,1.0,MSFT
55,2022-10-12,225.40,227.86,223.960,225.75,224.064538,21903937,0.0,1.0,MSFT
201,2022-03-15,190.80,192.28,185.911,192.03,192.030000,31721682,0.0,1.0,META
93,2022-08-18,174.34,175.76,171.860,174.66,174.660000,18844451,0.0,1.0,META
469,2021-02-22,128.01,129.72,125.600,126.00,124.329736,102886922,0.0,1.0,AAPL
223,2022-02-10,228.27,235.00,226.700,228.07,228.070000,49310356,0.0,1.0,META


In [27]:
# Drop the specified columns
df_stocks = df_stocks.drop(columns=[ "Split_ration", "Dividend Amount"])

In [28]:
#Naming date column 
df_stocks.replace('Unnamed: 0', 'date')

,Unnamed: 0,open_price,highest_price,low_price,close_price,adjusted_close,volume,symbol
0,2022-12-30,128.41,129.9500,127.4300,129.93,129.552234,77034209,AAPL
1,2022-12-29,127.99,130.4814,127.7300,129.61,129.233164,75703710,AAPL
2,2022-12-28,129.67,131.0275,125.8700,126.04,125.673544,85438391,AAPL
3,2022-12-27,131.38,131.4100,128.7200,130.03,129.651943,69007830,AAPL
4,2022-12-23,130.92,132.4150,129.6400,131.86,131.476623,63814893,AAPL
...,...,...,...,...,...,...,...,...
498,2021-01-08,268.31,268.9500,263.1800,267.57,267.570000,18528251,META
499,2021-01-07,265.90,271.6100,264.7800,268.74,268.740000,15789756,META
500,2021-01-06,262.00,267.7500,260.0100,263.31,263.310000,24354149,META
501,2021-01-05,268.29,272.4000,268.2111,270.97,270.970000,9871557,META


In [29]:
df_stocks.columns = ['date', "open_price", "highest_price", "low_price", "close_price", "adjusted_close", "volume", 'symbol']

In [30]:
df_stocks

,date,open_price,highest_price,low_price,close_price,adjusted_close,volume,symbol
0,2022-12-30,128.41,129.9500,127.4300,129.93,129.552234,77034209,AAPL
1,2022-12-29,127.99,130.4814,127.7300,129.61,129.233164,75703710,AAPL
2,2022-12-28,129.67,131.0275,125.8700,126.04,125.673544,85438391,AAPL
3,2022-12-27,131.38,131.4100,128.7200,130.03,129.651943,69007830,AAPL
4,2022-12-23,130.92,132.4150,129.6400,131.86,131.476623,63814893,AAPL
...,...,...,...,...,...,...,...,...
498,2021-01-08,268.31,268.9500,263.1800,267.57,267.570000,18528251,META
499,2021-01-07,265.90,271.6100,264.7800,268.74,268.740000,15789756,META
500,2021-01-06,262.00,267.7500,260.0100,263.31,263.310000,24354149,META
501,2021-01-05,268.29,272.4000,268.2111,270.97,270.970000,9871557,META


In [31]:
#Save stock data to a CSV file
filename = f'stocks_data.csv'
df_stocks.to_csv(filename)

**Create SQLite Database: Install SQLite in your machine and create a database called etl_extended_case**

In [32]:
%%capture
!pip install ipython-sql sqlalchemy pandas
import sqlalchemy
sqlalchemy.create_engine("sqlite:///etl_extended_case.db'.db")
%load_ext sql
%sql sqlite:///etl_extended_case.db'.db

In [33]:
import sqlite3

# Create a connection to the database (this will create the database file if it doesn't exist)
conn = sqlite3.connect('etl_extended_case.db')

In [34]:
# Create a cursor object to execute SQL queries
cursor = conn.cursor()

In [36]:
#Creating table table for loading headline
headlines = cursor.execute('CREATE TABLE IF NOT EXISTS headlines_data (headline VARCHAR, sentiment_score FLOAT, sentiment VARCHAR(100), relevant_words VARCHAR, headline_spanish VARCHAR, headline_Italian VARCHAR, url VARCHAR, timestamp TIMESTAMP)')
conn.commit()

#Creating table for loading stocks data
stocks = cursor.execute('CREATE TABLE IF NOT EXISTS stocks_data(date TIME,  open_price FLOAT, highest_price FLOAT, low_price FLOAT, low_price FLOAT, close price FLOAT, adjusted_close FLOAT, volume float, symbol float)')
conn.commit()



In [37]:
#conn.execute(create_table_query)

# Load data from a CSV file into a pandas DataFrame
headlines_data = pd.read_csv('headlines_data.csv')

# Insert data into the SQLite table
headlines_data.to_sql('headlines', conn, if_exists='append', index=False)
#stocks_data.to_sql('stocks', conn, if_exists='append', index=False)

# Load data from stocks_data.csv into a pandas DataFrame
stocks_data = pd.read_csv('stocks_data.csv')

# Insert data into the stocks table
stocks_data.to_sql('stocks', conn, if_exists='append', index=False)

# Close the database connection
conn.close()

In [ ]:
import psycopg2

# Establish a connection to the Redshift cluster
conn = psycopg2.connect(
    host='localhost',
    port='5432',
    user='enochagbey',
    password='Hope7709',
    database='redshift'


# Create a cursor object to execute SQL commands
cur = conn.cursor()

# Define the schema name
schema_name = 'my_schema'

# Create the schema
create_schema_query = f"CREATE SCHEMA {payments_table};"
payments_table =cur.execute('CREATE TABLE IF NOT EXISTS payments_table (date_of_payment DATE, number_of_payments_included_in_total_amount INT, total_amount_of_payment_usdollars FLOAT, record_id INT)')
conn.commit()
                            
create_schema_query = f"CREATE SCHEMA {physicians_table};"
physicians_table =cur.execute('CREATE TABLE IF NOT EXISTS physicians_table (product_category_or_therapeutic_area_1 VARCHAR(50), covered_recipient_profile_id INT, record_id INT,  covered_recipient_npi INT physician_specialty VARCHAR(100),physician_first_name VARCHAR(100) , physician_last_name VARCHAR(100) , physician_middle_name VARCHAR(100))')


create_schema_query = f"CREATE SCHEMA {reviews_table};"
reviews_table =cur.execute('CREATE TABLE IF NOT EXISTS reviews_table (npi INT, source VARCHAR, quality_category_score FLOAT, pi_category_score INT,ia_category_score INT,final_mips_score_without_cpb FLOAT)')


# Commit the changes
conn.commit()

# Close the cursor and connection
cur.close()
conn.close()

In [ ]:

# Define the table name and data file location
table_name1 = 'payments_table'
data_file = '/path/to/data/file.csv'

table_name2 = 'physicians_table'
data_file = '/path/to/data/file.csv'

table_name3 = 'reviews_table'
data_file = '/path/to/data/file.csv'


data_file = '/path/to/data/file.csv'

# Define the COPY command
copy_command = f"COPY {table_name} FROM '{data_file}' DELIMITER ',' CSV;"

# Execute the COPY command
cur.execute(copy_command)

# Commit the changes
conn.commit()

# Close the cursor and connection
cur.close()
conn.close()